![lsbu](https://storage-prtl-co.imgix.net/endor/organisations/11506/logos/1540893470_rsz_lsbu_crest_simple_white_text_horizon_blackoutline.jpg)

## Introduction
This notebook shows how to configure the flask application, implementing the `Attendance monitoring via SMS` project. 

## Objectives
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

## Configuartion

Make sure you have the required environments and packages configured, as mentioned in the `01a_set_up_the_environments
` notebook.

## Import Libraries

In [ ]:
import os
import re
import pathlib
import secrets
import pandas as pd
from typing import Dict, Optional
from flask import Flask, request, render_template, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from bs4 import BeautifulSoup
import altair as alt
from flask_mail import Mail, Message
from dotenv import load_dotenv
from datetime import date, datetime
from twilio.rest import Client

## Folder Structure
```python
|-- attendance_monioring_via_sms
|   |-- .twilio
|   |-- datasets
|      |-- *.csv
|   |-- docs
|      |-- *.md
|      |-- *.pdf
|   |-- env_vars
|      |-- .env.txt
|   |-- images
|      |-- *.png
|   |-- nbs
|      |-- *.ipynb 
|   |-- static
|      |- *.css
|      |- *.js
|   |-- templates
|      |- *.html
|   |-- app.py
|   |-- conda_env_twilio.yml
|   |-- requirements.txt

```

## Set Paths

In [ ]:
# Set paths
base_dir = pathlib.Path().absolute().parent

# Path to all data directories
data = pathlib.Path(base_dir/'datasets')

# Path to credentials data
credentials = data/'credentials'

# Path to raw data
raw = data/'raw'

# Path to processed data
processed = data/'processed'

# Path to environment variables
env_vars = pathlib.Path(base_dir/'env_vars')

## Load Environment Vaiables
All the sensitive user information are stored in a `.env.txt` file, which is loaded, using `dotenv`.

In [ ]:
# Load dotenv file
load_dotenv(env_vars/'.env.txt')

## Configure Flask App
The flask app has a flask-mail app to send emails, which is configured with an `smtp` based mail testing software/website called, `Mailtrap`. It creats dummy emails, sender and receiver rather than acutally sending the emails but with the same conventions that apply to the real emails.

The flask_login app tracks the login of the users to the application and used for session management and provide the endpoints from unauthorised access by an adversary.

In [ ]:
# Flask app
app = Flask(__name__)
# Configure a secret-key for the flask app
app.config["SECRET_KEY"] = secrets.token_hex(24)

# Configure flask mail app
app.config['MAIL_SERVER'] = 'smtp.mailtrap.io'
app.config['MAIL_PORT'] = 2525
app.config['MAIL_USERNAME'] = os.environ.get('MAIL_USERNAME')
app.config['MAIL_PASSWORD'] = os.environ.get('MAIL_PASSWORD')
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False

# Create login_manager app
login_manager = LoginManager(app)
# Create mail app
mail = Mail(app)

# Create twilio client object
account_sid = os.environ.get("TWILIO_ACCOUNT_SID")
auth_token = os.environ.get("TWILIO_AUTH_TOKEN")
client = Client(account_sid, auth_token)

# Define a User class to store all the users from the .csv input files
users: Dict[str, "User"] = {}

class User(UserMixin):
    def __init__(self, id: str, username: str, email: str, password: str):
        self.id = id
        self.username = username
        self.email = email
        self.password = password

    @staticmethod
    def get(user_id: str) -> Optional["User"]:
        return users.get(user_id)

    def __str__(self) -> str:
        return f"<Id: {self.id}, Username: {self.username}, Email: {self.email}>"

    def __repr__(self) -> str:
        return self.__str__()

data = pd.read_csv(credentials/'login_credentials.csv')
for index in data.index:
    users[str(index)] = User(
        id=index,
        username=data.loc[index, "first_name"],
        email=data.loc[index, "email"],
        password=data.loc[index, "password"],
    )

###
"""
All the endpoints for the application goes here
"""
###

if __name__ == '__main__':
    app.run(debug=True)

## Endpoint for Showing Attendance

In [ ]:
# Endpoint for showing attendance
@app.route('/check_attendance', methods=['GET', 'POST'])
@login_required
def check_attendance():  
    message = None
    df_courses = pd.read_csv(credentials/'courses.csv')
    df_timetable = pd.read_csv(credentials/'timetable.csv', parse_dates=['week_start', 'week_end'], infer_datetime_format=True)
    courses = df_courses.course_name.dropna().unique()
    weeks = list(df_timetable.academic_week.dropna().unique())

    if request.method == 'POST':
        course_name =request.form['comp_select']
        course_code = df_courses.loc[df_courses.course_name == course_name, 'course_code'].squeeze()
        semester = int(request.form['comp_select1'])
        academic_week = int(request.form['comp_select2'])

        file_name = f'{course_code}_semester_{semester}_week_{academic_week}_attendance.csv'

        if (processed/'attendance'/file_name).exists():
        
            df = pd.read_csv(processed/'attendance'/file_name).dropna()
            headings = df.columns.values
            headings = tuple(headings)

            data = df.to_records(index=False)
            data = tuple(data)
            return render_template('table6.html', headings=headings, data=data, course_name=course_name, semester=semester, academic_week=academic_week)
        else:
            message = f"No attendance found for '{course_name}', 'semester {semester}', 'week {academic_week}'! Please go to the 'Create Attendance' page to create an attendance."
            return render_template('check_attendance.html', message=message, courses=courses, weeks=weeks)
    return render_template('check_attendance.html', message=message, courses=courses, weeks=weeks)

## Endpoint for Showing Spammed Attendance

In [ ]:
# Endpoint for showing spammed attendance
@app.route('/check_spams', methods=['GET', 'POST'])
@login_required
def check_spams():  
    message = None
    df_courses = pd.read_csv(credentials/'courses.csv')
    df_timetable = pd.read_csv(credentials/'timetable.csv', parse_dates=['week_start', 'week_end'], infer_datetime_format=True)
    courses = df_courses.course_name.dropna().unique()
    weeks = list(df_timetable.academic_week.dropna().unique())

    if request.method == 'POST':
        course_name =request.form['comp_select']
        course_code = df_courses.loc[df_courses.course_name == course_name, 'course_code'].squeeze()
        semester = int(request.form['comp_select1'])
        academic_week = int(request.form['comp_select2'])

        file_name_spam = f'{course_code}_semester_{semester}_week_{academic_week}_spammed.csv'

        if (processed/'spam'/file_name_spam).exists():
        
            df = pd.read_csv(processed/'spam'/file_name_spam).dropna(how='all')
            headings = df.columns.values
            headings = tuple(headings)

            data = df.to_records(index=False)
            data = tuple(data)
            return render_template('table7.html', headings=headings, data=data, course_name=course_name, semester=semester, academic_week=academic_week)
        else:
            message = f"No spams found for '{course_name}', 'semester {semester}', 'week {academic_week}'! Please go to the 'Create Attendance' page to create an attendance."
            return render_template('check_spams.html', message=message, courses=courses, weeks=weeks)
    return render_template('check_spams.html', message=message, courses=courses, weeks=weeks)

## Endpoint for Displaying Phone Number

In [ ]:
# Endpoint for displaying phone number
@app.route('/display_number', methods=['GET', 'POST'])
@login_required
def display_number():  
    message = None
    df_courses = pd.read_csv(credentials/'courses.csv')
    courses = df_courses.course_name.dropna().unique()
    number = os.environ.get('TWILIO_PHONE_NUMBER')
    if request.method == 'POST':
        course_name =request.form['comp_select']
        course_code = df_courses.loc[df_courses.course_name == course_name, 'course_code'].squeeze()
        start_time = df_courses.loc[df_courses.course_name == course_name, 'start_time'].squeeze()
        end_time = df_courses.loc[df_courses.course_name == course_name, 'end_time'].squeeze()
        duration = f'{start_time}-{end_time}'
        return render_template('display_number.html', message=message, courses=courses, course_code=course_code, duration=duration, number=number)
    return render_template('display_number.html', message=message, courses=courses)

## Endpoint for Sending Reminder

In [ ]:
# Endpoint for sending reminder
@app.route('/send_reminder', methods=['GET', 'POST'])
@login_required
def send_reminder():  
    message = None
    df_courses = pd.read_csv(credentials/'courses.csv')
    courses = df_courses.course_name.dropna().unique()
    number = os.environ.get('TWILIO_PHONE_NUMBER')
    if request.method == 'POST':
        course_name =request.form['comp_select']
        course_code = df_courses.loc[df_courses.course_name == course_name, 'course_code'].squeeze()
        start_time = df_courses.loc[df_courses.course_name == course_name, 'start_time'].squeeze()
        end_time = df_courses.loc[df_courses.course_name == course_name, 'end_time'].squeeze()
        duration = f'{start_time}-{end_time}'

        file_name = f'{course_code}_enrolled_students.csv'
        
        df_students = pd.read_csv(credentials/'enrolled'/file_name)
        for i in range(len(df_students)):
            msg = Message(f"Attendance Reminder '{course_name}'...",
                          sender="admin@attendance.lsbu.ac.uk", recipients=[df_students.email[i]])
            msg.body = f"Hi {df_students.first_name[i]},\n\nPlease kindly register your attendance for '{course_name}' between {duration}.\n\nPlease kindly quote: '{df_students.student_id[i]} {course_code}' in your SMS and send to '{number}.'\n\nYou will receive an acknowledgment later for your attendance.\n\nShould you require any assistance, please contact your tutor.\n\nKind Regards,\nAttendance Team"
            mail.send(msg)
        message = f"Reminder sent for '{course_name}!'" 
        return render_template('send_reminder.html', message=message, courses=courses)
    return render_template('send_reminder.html', message=message, courses=courses)

### Summary
In this notebook it was demonstrated how to:
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

<center><b>Author</b></center>

| Name | Date Created | Last Modified |
|------|--------------|---------------|
|Khaled Ahmed | 30/01/2023 | 30/01/2023|